In [1]:
import sys
sys.path.insert(0, 'analysator')
from analysator import pytools as pt

Using LaTeX formatting
Using backend module://matplotlib_inline.backend_inline
Using LaTeX formatting
Using backend module://matplotlib_inline.backend_inline
Note: Did not import (outdated MayaVi2) grid module:  No module named 'traits'
Using matplotlib version 3.4.3
Note: Did not import (outdated MayaVi2) grid module:  No module named 'traits'


In [2]:
filename = 'data/bulk.0000015.vlsv'

In [3]:
f = pt.vlsvfile.VlsvReader(file_name=filename)

Found population proton


In [4]:
f.list()
m = f.get_spatial_mesh_size()
print(m)
a = f.read_variable("vg_v")
print(a)
print(len(a))
t = f.read_parameter('time')
print(f't = {t}')

tag = PARAMETER
    time
    dt
    timestep
    fieldSolverSubcycles
    fileIndex
    xmin
    xmax
    ymin
    ymax
    zmin
    zmax
    xcells_ini
    ycells_ini
    zcells_ini
    version
    numWritingRanks
tag = VARIABLE
    CellID
    fg_b
    fg_e
    proton/vg_blocks
    proton/vg_rho_nonthermal
    proton/vg_v_nonthermal
    proton/vg_ptensor_nonthermal_diagonal
    proton/vg_ptensor_nonthermal_offdiagonal
    proton/vg_rho_thermal
    proton/vg_v_thermal
    proton/vg_ptensor_thermal_diagonal
    proton/vg_ptensor_thermal_offdiagonal
    proton/vg_ptensor_diagonal
    proton/vg_ptensor_offdiagonal
    proton/vg_rho
    proton/vg_v
    vg_b_vol
    vg_boundarytype
    vg_f_saved
    vg_rank
[1200    1    1]
[[-2.66698448e+05  4.34434731e+01 -1.31178814e+06]
 [-2.66699390e+05  4.72151292e+01 -1.31178692e+06]
 [-2.66699208e+05  4.66805985e+01 -1.31178649e+06]
 ...
 [-7.49629967e+05  3.72498840e+01 -1.29813111e+06]
 [-7.49633943e+05  3.30509251e+01 -1.29813487e+06]
 [-7.49638